## Denoising using Wiener Filter

### Problem Statement

Consider an image corrupted by both additive Gaussian noise and defocus blur. Design an algorithm to restore the image to make it less noisy and less blurry. Evaluate the restored image using appropriate quality metrics for varying amounts of noise and blur. Hint: One can use Wiener filter or its variants to achieve this.


### Introduction

#### About
For a signal d(n), possessing a noise n(n), we have effective signal as $x(n) = d(n)*h(n) + n(n)$ in the sample domain. Our aim is to reconstruct the signal d(n) from a given x(n). For this we use Wiener Filters.

Wiener Filter is an Adaptive Filter. It works based on the PSF of the functions; such that based on the training data it sees, the filter will find a $g(n)$ to convert $x(n)$ to reconstructed signal $s(n)$ to match $d(n)$. In frequency domain for PSD's D and N respectively, the transfer function corresponds to, 
$G(u,v) = E[\frac{H(u,v)^* D}{H(u,v)^* H(u,v) D + N}]$

Here, we take the assumption that each pixel from each image (data point) has equal contribution. Hence the pmf is uniform.

#### Modelling the Noise
Here, we will transform the image a gaussian kernel which is the transfer function of an LTI system. After this, we add noise to images. Wiener Filters are defined for WSS processes. For a AWGN, we can assume Gaussian Noise with a mean zero and variance randomly varying for the sake of robustness.

### Importing Libraries

The following libraries are needed. They can be installed by `pip install <lib-name>` or running `requirements.txt` attached along with this notebook.
1. cv2 for image read , write and display
2. os to access files from dataset
3. numpy for all operations on the image

In [1]:
import cv2
import os
import numpy as np
import scipy
from scipy.signal import convolve

### Approach

#### Metrics
We use two metrics, namely Peak Signal to Noise Ratio (PSNR) and Structural Similarity Index (SSIM)

In [2]:
def psnr(im1, im2):
    mse = np.mean((im1-im2)**2)
    if mse == 0:
        return 100
    return 20 * np.log10(255 / np.sqrt(mse))

In [3]:
def ssim(im1, im2):
    from skimage.measure import compare_ssim
    if im1.dtype!=im2.dtype:
        im1 = np.int64(im1)
        im2 = np.int64(im2)
    if im1.shape>1:
        return compare_ssim(im1, im2, multichannel=True)
    else:
        return compare_ssim(im1,im2)

### Preprocessing & Additions
Images are first, padded and then patched into 64 x 64 size. Then, the images are convolved with a gaussian blur and finally gaussian noise is added.

In [4]:
size = 64

def pad(im, w0=size, h0=size):
    w,h = im.shape
    l_pad = (w0-w)//2
    t_pad = (h0-h)//2

    if w%2==0 and h%2==0:
        im2 = cv2.copyMakeBorder(im, l_pad,t_pad,l_pad,t_pad,cv2.BORDER_CONSTANT,value=[0,0,0])

    elif w%2!=0 and h%2==0:
        im2 = cv2.copyMakeBorder(im, l_pad+1,l_pad,t_pad,t_pad,cv2.BORDER_CONSTANT,value=[0,0,0])

    elif w%2==0 and h%2!=0:
        im2 = cv2.copyMakeBorder(im, l_pad,l_pad,t_pad+1,t_pad,cv2.BORDER_CONSTANT,value=[0,0,0])
    else:
        im2 = cv2.copyMakeBorder(im, l_pad+1,l_pad,t_pad+1,t_pad,cv2.BORDER_CONSTANT,value=[0,0,0])
    return im2

In [5]:
def noise(row = size,col = size,ch=1):
    mean = 127
    var = np.random.randint(0,10)
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    return gauss

In [6]:
# Credits: https://www.scipy-lectures.org/intro/scipy/auto_examples/solutions/plot_image_blur.html

def blur(dim=5):
    t = np.linspace(-10, 10, dim)
    bump = np.exp(-0.1*t**2)
    bump /= np.trapz(bump) # normalize the integral to 1

    # make a 2-D kernel out of it
    kernel = bump[:, np.newaxis] * bump[np.newaxis, :]
    return kernel

#### Dataset

Training Dataset chosen is Set5 dataset, more because of the variety it captures in terms of scenes as well as colours. The data is loaded and padded to be then appended in the data variable. This will ensure uniformity in image size.
Patches are stored in basic as well as blurred form to create our training set.

In [7]:
train_folder = "./Data/Set5/"

files = os.listdir(train_folder)

images = []
blurred = []

for file_name in files:
    #print(files)
    name = file_name.split(".")[0]
    img = cv2.imread(train_folder+file_name,0)
    w,h = img.shape
    
    filt = blur()
    p = convolve(img,filt,'same')
    p = p/np.max(p)

    for i in range(size,w,size):
        for j in range(size,h+1,size):
            patch = img[(i-size):i,(j-size):j]/255.0
            images.append(patch)
            blurred.append(p[(i-size):i,(j-size):j])

In [8]:
blurred = np.array(blurred)
images = np.array(images)
print(blurred.shape, images.shape)

((115, 64, 64), (115, 64, 64))


#### Training
The training function is based on the fact that using Wiener Filter in frequency domain for PSD's D and N respectively, the transfer function corresponds to, 
$G(u,v) = E[\frac{H(u,v)^* D}{H(u,v)^* H(u,v) *D + N}]$

Finally $G(u,v)$ gives us the final filter coefficients

In [9]:
def train(blurred, images):
    G = np.zeros(images[0].shape, dtype='complex128')
    G1 = np.zeros(images[0].shape, dtype='complex128')
    G2 = np.zeros(images[0].shape, dtype='complex128')
    #avg = np.mean(data,axis=0)
    l = images.shape[0]
    for i in range(l):
        img = images[i]
        x = blurred[i]
        row, col = img.shape
        gauss = noise(row, col)/255
        #gauss = gauss/np.max(gauss)
        D = np.fft.fft2(img)
        N = np.fft.fft2(gauss)
        X = np.fft.fft2(x+gauss)
        H = (X-N)/D
        G1 = (np.conjugate(H)*D*np.conj(D))/65536
        G2 = (np.conjugate(H)*H*D*np.conj(D)/65536 + N*np.conj(N)/65536)
        G += G1/G2
    return G/l

In [10]:
H = train(images, blurred)
if np.real(np.max(H))>10000:
    H = train(images, blurred)
    
np.min(H), np.max(H), H.shape

((0.06035113853627861+0.04359942937012486j),
 (1.0196216143653307+0.00029008108831199636j),
 (64, 64))

### Visualizing H(u,v)
Transfer function should be a band pass filter and must be given by circle

In [11]:
k = np.abs(H)
k = k/np.max(k)
#display(k)
cv2.imwrite("./Results/Transfer.jpg",np.uint8(k*255))

True

### Testing

For testing we will first define a function to display an image. This is done to avoid redundancy in code. `display()` will internally work on `cv2.imshow()`

In [12]:
def display(image, name='Screen'):
    cv2.imshow(name,image)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [13]:
def test(im):
    #im = cv2.imread('/home/manas/Datasets/Dataset/BSDS300/images/train/134008.jpg',0)
    w,h = im.shape
    size = 64

    im = pad(im, int(np.ceil(w*1.0/size)*size), int(np.ceil(h*1.0/size)*size))
    final = np.zeros(im.shape)
    w,h = im.shape
    #print w,h
    gauss = noise(w,h)
    #gauss -= np.min(gauss)
    y = convolve(im,blur(),'same')
    imnoise = y + gauss
    imnoise[imnoise>255] = 255
    imnoise[imnoise<0] = 0
    imnoise /= 255

    for i in range(size,w+1,size):
            for j in range(size,h+1,size):
                patch = imnoise[(i-size):i,(j-size):j]
                
                X_test = np.fft.fft2(patch)
                out = np.fft.ifft2(H*X_test)
                output = np.absolute(out)
                output = output/np.max(output)
                final[(i-size):i,(j-size):j] = output
#     display(imnoise)
#     display(final)
    cv2.imwrite("./Results/"+name+"_Output.jpg",np.uint8(final*255))
    cv2.imwrite("./Results/"+name+"_Input.jpg",np.uint8(imnoise*255))
    return psnr(np.uint8(imnoise*255),im), psnr(im,np.uint8(final*255.0)), ssim(im,final), ssim(np.uint8(imnoise*255),im)

In [14]:
folder = "./Data/Set14/"

files = os.listdir(folder)
files.sort()

avg_psnr = 0
avg_ssim = 0

print "All data in form of PSNR in db and SSIM index"

for file_name in files:
    name = file_name.split(".")[0]
    img = cv2.imread(folder+file_name,0)
    result = test(img)
    avg_psnr += result[1]-result[0]
    avg_ssim += result[2]
    
    print ("Image: "+name+"\nNoise: "+str(result[0])+", "+str(result[3])+", Output: "+str(result[1])+", "+str(result[2]))
    
print "Average output values:",
print avg_psnr/len(files), avg_ssim/len(files)

All data in form of PSNR in db and SSIM index
Image: 1
Noise: 32.44446422098167, 0.5722711439548581, Output: 27.600519776289225, 1.0
Image: 10
Noise: 32.4076981265208, 0.5220048588899826, Output: 27.491359385853066, 1.0
Image: 11
Noise: 34.12002742794045, 0.6599827882986563, Output: 28.0099104352151, 1.0
Image: 12
Noise: 30.661732837589227, 0.6474724623132171, Output: 27.504855535254787, 1.0
Image: 13
Noise: 32.752958806282464, 0.6376579248677645, Output: 28.58281395558045, 1.0
Image: 14
Noise: 29.77376725581206, 0.37644199821123225, Output: 27.77227550315712, 1.0
Image: 2
Noise: 32.06950404364448, 0.5599221217470589, Output: 27.817173755701795, 1.0
Image: 3
Noise: 31.46500636848996, 0.5651699124148128, Output: 27.51593351271763, 1.0
Image: 4
Noise: 30.764460557569933, 0.4085695907121556, Output: 27.443430760166585, 1.0
Image: 5
Noise: 30.148075354025266, 0.4560245821733598, Output: 27.70481848716593, 1.0
Image: 6
Noise: 34.04607028958711, 0.3393041794131283, Output: 27.888245701906676

### Example Testing
We choose a random image to add noise to and the denoise it using Wiener Filter

In [15]:
im = cv2.imread('./Data/134052.jpg',0)
#test(im)
w,h = im.shape
size = 64

im = pad(im, int(np.ceil(w*1.0/size)*size), int(np.ceil(h*1.0/size)*size))
final = np.zeros(im.shape)
w,h = im.shape
gauss = noise(w,h)

y = convolve(im,blur(),'same')
imnoise = y + gauss
imnoise[imnoise>255] = 255
imnoise[imnoise<0] = 0
imnoise /= 255


for i in range(size,w+1,size):
        for j in range(size,h+1,size):
            patch = im[(i-size):i,(j-size):j]
            #print np.max(imnoise), np.min(imnoise), np.min(gauss), np.max(gauss)
            #print psnr(im,imnoise), ssim(im,imnoise)
            X_test = np.fft.fft2(patch)
            out = np.fft.ifft2(H*X_test)
            output = np.absolute(out)
            # output[output<0] = 0.0
            # output[output>1] = 1.0
            #np.max(output)
            output = output/np.max(output)
            final[(i-size):i,(j-size):j] = output
       

#### Exploration 
Checking for the PSNR and SSIM between image and the blurred image for the testing image, reveals us the amount of degradation brought in image based on noise.
After that the images are displayed for perceptual understanding.

In [16]:
display(im, "Original")
display(gauss/np.max(gauss), "Noise")
display(imnoise, "Noisy Image")
display(final,"Wiener Output")
cv2.imwrite("./Results/Test_In.jpg",np.uint8(imnoise*255))
cv2.imwrite("./Results/Test_Out.jpg",np.uint8(final*255))

True

#### Evaluation
The metrics are based on PSNR and SSIM of the image after filtering and the source image (noise free)

In [17]:
print "Noise"
print "PSNR: ", round(psnr(np.uint8(imnoise*255),im),4),"dB"
print "SSIM: ", round(ssim(im*1.0,imnoise*255.0),4)
print "\nOutput"
print "PSNR: ",round(psnr(np.uint8(final*255), im),4),"dB"
print "SSIM: ",round(ssim(im*1.0, final*255.0),4)

Noise
PSNR:  35.0845 dB
SSIM:  0.4025

Output
PSNR:  27.8148 dB
SSIM:  0.5478
